# RAG evaluation с RAGAS (Colab пример)

Этот ноутбук повторяет пример из заметки `01_ragas_eval_example` и позволяет локально в Colab:

- создать небольшой датасет для оценки RAG,
- настроить RAGAS для трёх метрик:
  - `Faithfulness`
  - `ContextRelevance`
  - `AnswerRelevancy`,
- запустить оценку и посмотреть результаты.

> ⚠️ **Важно:** ниже пример рассчитан на LLM через OpenAI API. Если хочешь использовать Ollama, смотри комментарий в соответствующей ячейке и поменяй `base_url` / ключ.


In [ ]:
# Установка зависимостей (Colab)
!pip install -q ragas openai sentence-transformers


## 1. Датасет для оценки

Используем несколько простых примеров "вопрос → ответ + контекст".


In [ ]:
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset

dataset = EvaluationDataset(
    samples=[
        SingleTurnSample(
            user_input="Какой инструмент используется для миграций базы данных?",
            response="Для миграций базы данных используется Alembic.",
            retrieved_contexts=[
                "Alembic используется для управления миграциями базы данных в SQLAlchemy.",
            ],
        ),
        SingleTurnSample(
            user_input="Что такое FastAPI?",
            response="FastAPI — это современный Python-фреймворк для создания API.",
            retrieved_contexts=[
                "FastAPI — это современный Python-фреймворк для создания API.",
            ],
        ),
    ]
)

len(dataset.samples), dataset.samples[0]


## 2. Настройка LLM и embedding-модели

### Вариант A: OpenAI API (gpt-4o-mini / gpt-4.1 и т.п.)

Нужно выставить переменную окружения `OPENAI_API_KEY` (в Colab: `os.environ["OPENAI_API_KEY"] = "sk-..."`).

### Вариант B: Ollama с OpenAI-совместимым API

Если ты запускаешь код локально и используешь Ollama:

```python
client = AsyncOpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
llm = llm_factory("mistral", client=client)
```


In [ ]:
import os
import asyncio
from openai import AsyncOpenAI
from ragas.llms import llm_factory
from ragas.embeddings import HuggingFaceEmbeddings

# === LLM ===
# В Colab по умолчанию используем OpenAI API (замени модель при необходимости).
# Убедись, что задан OPENAI_API_KEY (через переменные окружения или напрямую в коде).

client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))
llm = llm_factory("gpt-4o-mini", client=client)

# === Embeddings ===
embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

llm, embeddings


## 3. Метрики: Faithfulness, ContextRelevance, AnswerRelevancy


In [ ]:
from pydantic import BaseModel
from ragas.metrics.collections import (
    Faithfulness,
    AnswerRelevancy,
    ContextRelevance,
)


class Result(BaseModel):
    faithfulness: float
    answer_relevancy: float
    context_relevance: float


async def evaluate_sample(sample: SingleTurnSample) -> Result:
    """Оценка одной пары (вопрос, ответ, контекст) по трём метрикам."""

    # 1. Faithfulness — верность контексту
    faith = await Faithfulness(llm=llm).ascore(
        user_input=sample.user_input,
        response=sample.response,
        retrieved_contexts=sample.retrieved_contexts,
    )

    # 2. AnswerRelevancy — релевантность ответа вопросу
    ar = await AnswerRelevancy(
        llm=llm,
        embeddings=embeddings,
    ).ascore(
        user_input=sample.user_input,
        response=sample.response,
    )

    # 3. ContextRelevance — релевантность контекста запросу
    cr = await ContextRelevance(
        llm=llm,
        embeddings=embeddings,
    ).ascore(
        user_input=sample.user_input,
        retrieved_contexts=sample.retrieved_contexts,
    )

    return Result(
        faithfulness=faith.value,
        answer_relevancy=ar.value,
        context_relevance=cr.value,
    )


## 4. Запуск оценки


In [ ]:
async def main():
    tasks = [evaluate_sample(sample) for sample in dataset.samples]
    results = await asyncio.gather(*tasks)

    for i, res in enumerate(results):
        print(f"Sample {i + 1}: {res}")


await main()
